In [36]:
import torch
import torchvision
from torch import nn 
#from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image 
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader
import tqdm
import scipy.io as io
import numpy as np
import sys
sys.path.append('../')
import dataset_utils

In [37]:
beetle_dataset['DAT']

KeyError: 'DAT'

In [43]:
beetle_dataset = io.loadmat("standard_fish.mat")
beetle_dataset.keys()
DAT = beetle_dataset['DAT'][0]
boldids = []
for boldid in np.squeeze(DAT[5][0]):
    boldids.append(boldid[0])
nucleotides = []
for nucleotide in DAT[0].squeeze():
    #print(len(nucleotide[0]))
    nucleotides.append(nucleotide[0])

In [44]:
onehot_nucleotides = torch.tensor(np.array([dataset_utils.one_hot_encoding(x) for x in nucleotides]))

In [40]:
#all_dnas = torch.load('../tensor_dataset/all_dnas.pt')

In [45]:
bb = dict()
bb['onehots'] = onehot_nucleotides.numpy()
io.savemat('onehots_fishes.mat',bb)

In [23]:
class DNAdataset(Dataset):
    def __init__(self, data, targets):
        self.data = data.float()
        self.targets = None
        
    def __getitem__(self, index):
        x = self.data[index].unsqueeze(0)
        return x, -1
    def __len__(self):
        return len(self.data)
        
dataset_nucleotides = DNAdataset(onehot_nucleotides, None)


In [24]:
dataloader_nucleotides = DataLoader(dataset_nucleotides, batch_size=32,shuffle=False)


In [28]:
class TinyModel(torch.nn.Module):
    def __init__(self):
        super(TinyModel, self).__init__()

        self.conv1 = torch.nn.Conv2d(1,16,(5,1))
        self.activation1 = torch.nn.LeakyReLU()
        self.norm1 = torch.nn.BatchNorm2d(16)
        self.conv2 = torch.nn.Conv2d(16,1,(5,1))
        self.activation2 = torch.nn.LeakyReLU()
        self.norm2 = torch.nn.BatchNorm2d(1)
        self.dropout1= torch.nn.Dropout(0.70)
        self.flat = torch.nn.Flatten()
        self.linear = torch.nn.Linear(3250,1500)
        self.dropout2= torch.nn.Dropout(0.70)
        self.activation3 = torch.nn.LeakyReLU()
        self.linear2 = torch.nn.Linear(1500,1050)
        #self.softmax = torch.nn.Softmax()
    def forward(self, x):
        #print(x.shape)
        x = self.conv1(x)
        x = self.activation1(x)
        x = self.norm1(x)
        x = self.conv2(x)
        x = self.activation2(x)
        x = self.norm2(x)
        x = self.dropout1(x)
        x = self.flat(x)
        x = self.linear(x)
        x = self.dropout2(x)
        x = self.activation3(x)
        x = self.linear2(x)
        return x
    def feature_extract(self,x):
        x = self.conv1(x)
        x = self.activation1(x)
        x = self.norm1(x)
        x = self.conv2(x)
        x = self.flat(x)
        return x




model = TinyModel()
model.cuda()
 
optimizer = torch.optim.Adam(model.parameters(),weight_decay=1e-5)
n_params = dataset_utils.count_trainable_parameters(model);
print(n_params)

6452761


In [29]:
state_dict = torch.load('../checkpoints/CNN_DNA_weights_for_unseen')
model.load_state_dict(state_dict['model_state_dict'])
optimizer.load_state_dict(state_dict['optimizer_state_dict'])

In [30]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.eval()
with torch.no_grad():
    features = []
    labels = np.array([]) 
    for dnas,batch_labels in dataloader_nucleotides:
        #print(dnas.shape)
        dnas = dnas
        dnas = dnas.to(device)
        fts = model.feature_extract(dnas)
        labels = np.concatenate((labels, batch_labels.cpu().numpy()))
        features.append(fts.cpu().numpy())
        torch.cuda.empty_cache()
    features = torch.tensor(np.concatenate(features))
    labels = torch.tensor(labels)


# Save extracted features in .mat

In [31]:
beetle_dataset = dict()
beetle_dataset['all_dna_features_cnn_new'] = features
io.savemat('fish_features.mat',beetle_dataset)


In [34]:
io.loadmat('fish_features.mat')['all_dna_features_cnn_new'].shape

(1070, 3250)